In [12]:
import pandas as pd
import pyodbc

# -------------------------
# SQL Server connection
# -------------------------
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)

# -------------------------
# Client list
# -------------------------
x = [
    'MarsManagement'
]

# -------------------------
# Store summary info
# -------------------------
summary_data = []

# -------------------------
# Fetch data per client
# -------------------------
for client in x:
    query = f"""
        SELECT *,
               FORMAT(TRY_CONVERT(date, Date, 105), 'dd') AS DayNum,
               CONCAT(UserName, '-', FORMAT(TRY_CONVERT(date, Date, 105), 'dd')) AS UserKey
        FROM {client}.ISPAttendanceMasterExport
        WHERE Month = '1' AND Year = '2026';
    """

    df = pd.read_sql(query, conn)

    # Clean column names
    df.columns = df.columns.str.strip()

    

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_27728\2515869420.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 795 entries, 0 to 794
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ISPAttendanceMasterExportId  795 non-null    int64 
 1   Month                        795 non-null    object
 2   Year                         795 non-null    object
 3   Region                       795 non-null    object
 4   State                        795 non-null    object
 5   City                         795 non-null    object
 6   RSMCode                      0 non-null      object
 7   BranchCode                   0 non-null      object
 8   ASMCode                      0 non-null      object
 9   SOCode                       0 non-null      object
 10  SupervisorCode               728 non-null    object
 11  ISPCode                      728 non-null    object
 12  EmployeeId                   771 non-null    object
 13  UserName                     795 no

In [14]:
import numpy as np

status_map = {
    "Present": "P",
    "Absent": "A",
    "Week Off": "WO",
    "National Holiday": "H",
    "Regional Holiday": "H",
    "Leave": "L",
    "Compensatory Off": "P"
}

df["Att_Code"] = df["Status"].map(status_map)



In [18]:
horizontal_df = df.pivot_table(
    index="UserName",
    columns="Date",
    values="Att_Code",
    aggfunc="first"
).reset_index()

horizontal_df.columns = [
    col.strftime("%d-%m-%Y") if isinstance(col, pd.Timestamp) else col
    for col in horizontal_df.columns
]



In [19]:
horizontal_df

UserName 01-01-2026 00:00:00 02-01-2026 00:00:00  \
0    ISPMARSGT-0001                   H                   P   
1    ISPMARSGT-0002                   H                   P   
2    ISPMARSGT-0003                   H                   P   
3    ISPMARSGT-0006                   H                   P   
4    ISPMARSGT-0007                   H                   P   
..              ...                 ...                 ...   
109    MARSGT-SUP-5                   H                   P   
110    MARSGT-SUP-6                   H                   P   
111  MARSGT-Test-01                   A                   A   
112  MARSGT-Test-02                   A                   A   
113  MarsGT-Test-03                 NaN                 NaN   

    03-01-2026 00:00:00 04-01-2026 00:00:00 05-01-2026 00:00:00  \
0                     P                  WO                   P   
1                     P                  WO                   P   
2                     P                  WO                   P   
3                     P                  WO                   P   
4                     P                  WO                   P   
..                  ...                 ...                 ...   
109                   P                  WO                   P   
110                   P                  WO                   P   
111                   A                   A                   P   
112                   A                   A                   A   
113                 NaN                 NaN                   A   

    06-01-2026 00:00:00 07-01-2026 00:00:00  
0                     P                   P  
1                     P                   P  
2                     P                   P  
3                     P                   P  
4                     P                   P  
..                  ...                 ...  
109                   P                   P  
110                   P                   P  
111                   A                   A  
112                   A                   A  
113                   A                   A  

[114 rows x 8 columns]

In [20]:
df1= pd.read_excel('MARS Attendance P1.xlsx',sheet_name="Attendance")

In [21]:
df_final = pd.merge(
    df1,
    horizontal_df,
    left_on="MR Code",     # column in df1
    right_on="UserName",   # column in df
    how="left"
)

In [22]:
df_final.to_excel('Final.xlsx', index=False)


In [23]:
horizontal_df.to_excel('SQL_DATA.xlsx', index=False)
